In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, layers

In [3]:
inputs = layers.Input(shape=(128,128,3), name="Input_Layer")

#Initial block
x = layers.Conv2D(
    filters = 32,
    kernel_size = (3,3),
    padding = 'same',
    use_bias = False,
    name = "Initial_Conv_Block"
)(inputs)

x = layers.BatchNormalization(name="Initial_Batchnorm")(x)
x = layers.LeakyReLU(alpha=0.01,name="Initial_Leaky")(x)

c:\Users\Vivekananda Katakam\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [4]:
# Residual Block
skip = x

x = layers.Conv2D(
    filters = 32,
    kernel_size = (3,3),
    padding = 'same',
    use_bias = False,
    name = "Residual_Conv_Block"
)(x)

x = layers.BatchNormalization(name="Residual_Batchnorm")(x)
x = layers.LeakyReLU(alpha=0.01,name="Residual_Leaky")(x)

x = layers.Conv2D(
    filters = 32,
    kernel_size = (3,3),
    padding = 'same',
    use_bias = False,
    name = "Residual2_Conv_Block"
)(x)

x = layers.BatchNormalization(name="Residual2_Batchnorm")(x)
x = layers.Add(name = "Added_Layer")([x,skip])
x = layers.LeakyReLU(alpha=0.01, name = "Residual2_Leaky")(x)

In [5]:
# Branch 1
b1 = layers.Conv2D(
    filters = 64,
    kernel_size = (3,3),
    padding = 'same',
    dilation_rate = 1,
    use_bias = False,
    name = "Branch1_Conv_Block"
)(x)
b1 = layers.LeakyReLU(alpha=0.01,name="Branch1_Leaky")(b1)

# Branch 2
b2 = layers.Conv2D(
    filters = 64,
    kernel_size = (3,3),
    padding = 'same',
    dilation_rate = 2,
    use_bias = False,
    name = "Branch2_Conv_Block"
)(x)
b2 = layers.LeakyReLU(alpha=0.01,name="Branch2_Leaky")(b2)

# Branch 3
b3 = layers.Conv2D(
    filters = 64,
    kernel_size = (3,3),
    padding = 'same',
    dilation_rate = 3,
    use_bias = False,
    name = "Branch3_Conv_Block"
)(x)
b3 = layers.LeakyReLU(alpha=0.01,name="Branch3_Leaky")(b3)

x = layers.Concatenate(axis=-1,name="Concat_layer")([b1, b2, b3])

In [6]:
# BottleNeck Layer
x = layers.Conv2D(
    filters=64,
    kernel_size=(1,1),
    padding='same',
    name="Bottleneck_Conv2d_Block",
    use_bias = False
)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)

In [7]:
# Dual Pooling
# Branch A
ba = layers.AveragePooling2D(
    strides = 2,
    pool_size=(2,2),
    name = "BranchA_AvgPool"
)(x)

bb = layers.MaxPooling2D(
    strides = 2,
    pool_size = (2,2),
    name = "BranchB_MaxPool"
)(x)

x = layers.Add(name="Added_Pooling_layer")([ba,bb])

In [8]:
model = Model(inputs = inputs, outputs = x )

In [9]:
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_Layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Initial_Conv_Block  │ (None, 128, 128,  │        864 │ Input_Layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Initial_Batchnorm   │ (None, 128, 128,  │        128 │ Initial_Conv_Blo… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Initial_Leaky       │ (None, 128, 128,  │          0 │ Initial_Batchnor… │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Residual_Conv_Block │ (None, 128, 128,  │      9,216 │ Initial_Leaky[0]… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Residual_Batchnorm  │ (None, 128, 128,  │        128 │ Residual_Conv_Bl… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Residual_Leaky      │ (None, 128, 128,  │          0 │ Residual_Batchno… │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Residual2_Conv_Blo… │ (None, 128, 128,  │      9,216 │ Residual_Leaky[0… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Residual2_Batchnorm │ (None, 128, 128,  │        128 │ Residual2_Conv_B… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Added_Layer (Add)   │ (None, 128, 128,  │          0 │ Residual2_Batchn… │
│                     │ 32)               │            │ Initial_Leaky[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Residual2_Leaky     │ (None, 128, 128,  │          0 │ Added_Layer[0][0] │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Branch1_Conv_Block  │ (None, 128, 128,  │     18,432 │ Residual2_Leaky[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Branch2_Conv_Block  │ (None, 128, 128,  │     18,432 │ Residual2_Leaky[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Branch3_Conv_Block  │ (None, 128, 128,  │     18,432 │ Residual2_Leaky[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Branch1_Leaky       │ (None, 128, 128,  │          0 │ Branch1_Conv_Blo… │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Branch2_Leaky       │ (None, 128, 128,  │          0 │ Branch2_Conv_Blo… │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Branch3_Leaky       │ (None, 128, 128,  │          0 │ Branch3_Conv_Blo

 Total params: 87,520 (341.88 KB)

 Trainable params: 87,200 (340.62 KB)

 Non-trainable params: 320 (1.25 KB)

None
